### Import libraries.

In [1946]:
import pandas as pd
import numpy as np

In [1947]:
%run functions.py

### Read the data.

In [1948]:
df_train = pd.read_csv('../datasets/train.csv')
df_test = pd.read_csv('../datasets/test.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Peak at the data.

In [1949]:
df_train.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA,Ex,Y,SBrkr,913,1209,0,2122,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997.0,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA,TA,Y,SBrkr,1057,0,0,1057,1.0,0.0,1,0,3,1,Gd,5,Typ,0,NaN,Detchd,1953.0,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,2Story,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA,Gd,Y,SBrkr,744,700,0,1444,0.0,0.0,2,1,3,1,TA,7,Typ,0,NaN,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1.5Fin,6,8,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,PConc,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,676.0,GasA,TA,Y,SBrkr,831,614,0,1445,0.0,0.0,2,0,3,1,TA,6,Typ,0,NaN,Detchd,1957.0,Unf,2.0,484.0,TA,TA,N,0,59,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [1950]:
df_train.shape

(2051, 81)

### Lower-case and eliminate spaces from column names.

In [1951]:
df_train.columns = ren_col(list(df_train.columns))
df_test.columns = ren_col(list(df_test.columns))

### Drop columns/rows with more than 70% of the values missing.

In [1952]:
# which columns are missing a lot of values
df_train.isnull().sum()[df_train.isnull().sum() > 0].sort_values(ascending=False)[:10]

pool_qc          2042
misc_feature     1986
alley            1911
fence            1651
fireplace_qu     1000
lot_frontage      330
garage_yr_blt     114
garage_cond       114
garage_qual       114
garage_finish     114
dtype: int64

 [Towards Data Science](https://towardsdatascience.com/feature-engineering-for-machine-learning-3a5e293a5114)

In [1953]:
# columns where more than 70% of the values are null/nan
threshold = 0.7
df_train[df_train.columns[df_train.isnull().mean() > threshold]].columns

Index(['alley', 'pool_qc', 'fence', 'misc_feature'], dtype='object')

In [1954]:
# drop the alley, pool_qc, fence, and misc_feature columns
df_train = df_train[df_train.columns[df_train.isnull().mean() < threshold]]

In [1955]:
df_train.shape

(2051, 77)

In [1956]:
# rows where more than 70% of the values are null/nan
df_train.loc[df_train.isnull().mean(axis=1) > threshold]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold,sale_type,saleprice


### Potential features.

In [1957]:
features = ['ms_zoning', 'lot_area', 'condition_1', 'overall_qual', 
            'overall_cond', 'year_built', 'year_remod/add', 'exter_qual',
            'exter_cond',  'bsmt_qual', 'bsmt_cond', 'bsmt_unf_sf', 
            'total_bsmt_sf', 'central_air', '1st_flr_sf', 'gr_liv_area',
            'full_bath', 'bedroom_abvgr', 'kitchen_abvgr', 'kitchen_qual',
            'fireplaces', 'garage_cars', 'garage_area', 'garage_qual', 
            'garage_cond', 'paved_drive', 'yr_sold']

### Remove all NaNs for the potential features.

In [1958]:
df_train.dropna(subset=features, inplace=True)
df_train[features].isnull().sum()[df_train[features].isnull().sum() > 0]

Series([], dtype: int64)

In [1959]:
df_train.shape

(1891, 77)

### Drop rows where the sale was commercial, industrial, or agricultural type.

In [1960]:
# we are interested in predicting sale price for homes, residential
df_train['ms_zoning'].value_counts()

RL         1495
RM          273
FV          101
RH           11
C (all)      11
Name: ms_zoning, dtype: int64

In [1961]:
# drop rows where the sale was commercial, industrial, or agricultural type
df_train = df_train[df_train['ms_zoning'] != 'C (all)']
df_train = df_train[df_train['ms_zoning'] != 'I (all)']
df_train = df_train[df_train['ms_zoning'] != 'A (agr)']

In [1962]:
df_train.shape

(1880, 77)

### Map string columns to numeric values.

In [1963]:
rating = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1}
yes_no = {'Y':1, 'N':0, 'P':2}
zoning = {'RL':1, 'RM':2, 'FV':3, 'RH':4}
condition = {'Norm':1, 'Feedr':2, 'Artery':3, 'RRAn':4,
             'PosN':5, 'RRAe':6, 'PosA':7, 'RRNn':8, 'RRNe':9}

In [1964]:
df_train['ms_zoning'] = df_train['ms_zoning'].map(zoning)
df_train['exter_qual'] = df_train['exter_qual'].map(rating);
df_train['exter_cond'] = df_train['exter_cond'].map(rating);
df_train['bsmt_qual'] = df_train['bsmt_qual'].map(rating);
df_train['bsmt_cond'] = df_train['bsmt_cond'].map(rating);
df_train['kitchen_qual'] = df_train['kitchen_qual'].map(rating);
df_train['central_air'] = df_train['central_air'].map(yes_no);
df_train['garage_qual'] = df_train['garage_qual'].map(rating);
df_train['garage_cond'] = df_train['garage_cond'].map(rating);
df_train['paved_drive'] = df_train['paved_drive'].map(yes_no);
df_train['condition_1'] = df_train['condition_1'].map(condition);

In [1965]:
df_test['ms_zoning'] = df_test['ms_zoning'].map(zoning)
df_test['exter_qual'] = df_test['exter_qual'].map(rating);
df_test['exter_cond'] = df_test['exter_cond'].map(rating);
df_test['bsmt_qual'] = df_test['bsmt_qual'].map(rating);
df_test['bsmt_cond'] = df_test['bsmt_cond'].map(rating);
df_test['kitchen_qual'] = df_test['kitchen_qual'].map(rating);
df_test['central_air'] = df_test['central_air'].map(yes_no);
df_test['garage_qual'] = df_test['garage_qual'].map(rating);
df_test['garage_cond'] = df_test['garage_cond'].map(rating);
df_test['paved_drive'] = df_test['paved_drive'].map(yes_no);
df_test['condition_1'] = df_test['condition_1'].map(condition);

In [1966]:
df_train[features].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1880 entries, 0 to 2050
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ms_zoning       1880 non-null   int64  
 1   lot_area        1880 non-null   int64  
 2   condition_1     1880 non-null   int64  
 3   overall_qual    1880 non-null   int64  
 4   overall_cond    1880 non-null   int64  
 5   year_built      1880 non-null   int64  
 6   year_remod/add  1880 non-null   int64  
 7   exter_qual      1880 non-null   int64  
 8   exter_cond      1880 non-null   int64  
 9   bsmt_qual       1880 non-null   int64  
 10  bsmt_cond       1880 non-null   int64  
 11  bsmt_unf_sf     1880 non-null   float64
 12  total_bsmt_sf   1880 non-null   float64
 13  central_air     1880 non-null   int64  
 14  1st_flr_sf      1880 non-null   int64  
 15  gr_liv_area     1880 non-null   int64  
 16  full_bath       1880 non-null   int64  
 17  bedroom_abvgr   1880 non-null   i

### Create new columns to evaluate age, quality, years since remodeling, and basement rating and area.

In [1967]:
df_train['age'] = np.abs(df_train['yr_sold'] - df_train['year_built'])
df_train['quality'] = df_train['exter_qual'] * df_train['exter_cond'] * df_train['overall_qual'] * df_train['overall_cond']
df_train['remod'] = np.abs(df_train['yr_sold'] - df_train['year_remod/add'])
df_train['bsmt_rating'] = df_train['bsmt_qual'] * df_train['bsmt_cond']
df_train['bsmt_sf'] = df_train['total_bsmt_sf'] - df_train['bsmt_unf_sf']

In [1968]:
df_test['age'] = np.abs(df_test['yr_sold'] - df_test['year_built'])
df_test['quality'] = df_test['exter_qual'] * df_test['exter_cond'] * df_train['overall_qual'] * df_train['overall_cond']
df_test['remod'] = np.abs(df_test['yr_sold'] - df_test['year_remod/add'])
df_test['bsmt_rating'] = df_test['bsmt_qual'] * df_test['bsmt_cond']
df_test['bsmt_sf'] = df_test['total_bsmt_sf'] - df_test['bsmt_unf_sf']

In [1969]:
features = ['ms_zoning', 'lot_area', 'condition_1', 'overall_qual', 
            'overall_cond', 'year_built', 'year_remod/add', 'exter_qual',
            'exter_cond',  'bsmt_qual', 'bsmt_cond', 'bsmt_unf_sf', 
            'total_bsmt_sf', 'central_air', '1st_flr_sf', 'gr_liv_area',
            'full_bath', 'bedroom_abvgr', 'kitchen_abvgr', 'kitchen_qual',
            'fireplaces', 'garage_cars', 'garage_area', 'garage_qual', 
            'garage_cond', 'paved_drive', 'yr_sold',
            'age', 'quality', 'remod', 'bsmt_rating', 'bsmt_sf']

### Impute nan values in the test dataset.

In [1970]:
df_test[features].isnull().sum()[df_test[features].isnull().sum() > 0]

ms_zoning       7
bsmt_qual      25
bsmt_cond      25
garage_qual    45
garage_cond    45
quality        75
bsmt_rating    25
dtype: int64

In [1971]:
df_test.fillna(df_test.mean(), inplace=True)

### Save the clean data into a new csv file.

In [1972]:
df_train_clean = df_train[features].copy()
df_train_clean['saleprice'] = df_train['saleprice'].copy()

df_test_clean = df_test[features].copy()

In [1973]:
df_train_clean.to_csv(r'../datasets/train_clean.csv', index=False)
df_test_clean.to_csv(r'../datasets/test_clean.csv', index=False)